In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import xgboost as xg
from xgboost import XGBRegressor


def one_hot_encode(
    df: pd.DataFrame,
    column: str,
    prefix: str,
) -> pd.DataFrame:

    df_ = df.copy()
    df_encoded = pd.get_dummies(df_[column], dtype="int")
    for col in df_encoded:
        df_[f"{prefix}_{col}"] = df_encoded[col]

    return df_.drop(columns=[column])

In [2]:
vuelos = pd.read_parquet("../resources/flights_clean.parquet")
ventas = pd.read_parquet("../resources/sales_clean.parquet")

In [3]:
# Merge a ambos dataframes
vuelos_ventas = pd.merge(ventas, vuelos, on="Flight_ID")
vuelos_ventas.drop(columns=["index", "Aeronave"], inplace=True)
vuelos_ventas

,Flight_ID,ProductType,ProductName,Quantity,TotalSales,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Semana,Mes
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
1,a05290288259526edd3601160b10e1de,Botanas,Cheetos,2,98.0,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
2,a05290288259526edd3601160b10e1de,Botanas,Ruffles Queso,6,294.0,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
3,a05290288259526edd3601160b10e1de,Refrescos,Coca Sin Azucar,2,96.0,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
4,a05290288259526edd3601160b10e1de,Licores,Jack And Coke,1,72.0,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095947,6113fc0448cd548cd7e7d3f76e7872c5,Perecederos,Cuerno Clasico De Pavo,5,575.0,AF,AW,Ciudad Principal,Ciudad Fronteriza,2023-11-02 23:10:00,2023-11-03 00:50:00,240,154.0,119.0,44,11
2095948,78b84bc241b924d6c89e171a34b0f60c,Perecederos,Cuerno Clasico De Pavo,1,115.0,AT,BP,MX Amigos y Familia,Ciudad Principal,2023-10-05 06:00:00,2023-10-05 07:05:00,240,188.0,150.0,40,10
2095949,2d7d30a16334ee2db84e5bc027a4a186,Licores,Vino Tinto Sangre De Toro,1,155.0,AO,BD,Playa,Ciudad Principal,2023-05-16 18:45:00,2023-05-16 19:35:00,180,149.0,112.0,20,5
2095950,7a56b895719a91ca6a98004cff956b7d,Licores,Vino Tinto Sangre De Toro,1,155.0,AL,AW,Ciudad Principal,MX Amigos y Familia,2023-02-15 07:00:00,2023-02-15 08:25:00,220,210.0,170.0,7,2


In [4]:
vuelos_ventas = vuelos_ventas.groupby("Flight_ID").agg(
    {
        "TotalSales": "sum",
        "Bookings": "mean",
        "Passengers": "mean",
        "Origin_Type": "first",
        "Destination_Type": "first",
        "STD": "first",
        "STA": "first",
        "DepartureStation": "first",
        "ArrivalStation": "first",
        "Semana": "first",
        "Mes": "first",
        "Capacity": "first",
    }
)
vuelos_ventas["STD"] = pd.to_datetime(vuelos_ventas["STD"])


vuelos_ventas["STA"] = pd.to_datetime(vuelos_ventas["STA"])


vuelos_ventas["Duracion"] = vuelos_ventas["STA"] - vuelos_ventas["STD"]


vuelos_ventas["Duracion"] = vuelos_ventas["Duracion"].dt.total_seconds() / 60


vuelos_ventas["Hora"] = vuelos_ventas["STD"].dt.hour


vuelos_ventas["Dia"] = vuelos_ventas["STD"].dt.day


# Cambiar tipos de destinos a one-hot encoding


destination_type = vuelos_ventas["Destination_Type"].map(
    {
        "Ciudad Principal": "destino_ciudadprincipal",
        "Playa": "destino_playa",
        "MX Amigos y Familia": "destino_amigosfamilia",
        "Ciudad Fronteriza": "destino_ciudadfronteriza",
        "Ecoturismo": "destino_ecoturismo",
    }
)


destination_type = pd.get_dummies(destination_type, dtype="int")


vuelos_ventas = (
    pd.concat([vuelos_ventas, destination_type], axis=1)
    if "Destination_Type" in vuelos_ventas.columns
    else vuelos_ventas
)


vuelos_ventas = (
    vuelos_ventas.drop(columns=["Destination_Type"])
    if "Destination_Type" in vuelos_ventas.columns
    else vuelos_ventas
)


origen_type = vuelos_ventas["Origin_Type"].map(
    {
        "Ciudad Principal": "origen_ciudadprincipal",
        "Playa": "origen_playa",
        "MX Amigos y Familia": "origen_amigosfamilia",
        "Ciudad Fronteriza": "origen_ciudadfronteriza",
        "Ecoturismo": "origen_ecoturismo",
    }
)


origen_type = pd.get_dummies(origen_type, dtype="int")


vuelos_ventas = (
    pd.concat([vuelos_ventas, origen_type], axis=1)
    if "Origin_Type" in vuelos_ventas.columns
    else vuelos_ventas
)


vuelos_ventas = (
    vuelos_ventas.drop(columns=["Origin_Type"])
    if "Origin_Type" in vuelos_ventas.columns
    else vuelos_ventas
)


vuelos_ventas["Book_Pass"] = vuelos_ventas["Bookings"] / vuelos_ventas["Passengers"]


vuelos_ventas.drop(columns=["STD", "STA"], inplace=True)


vuelos_ventas = one_hot_encode(vuelos_ventas, "DepartureStation", "departure")


vuelos_ventas = one_hot_encode(vuelos_ventas, "ArrivalStation", "arrival")

In [5]:
# Hacemos el modelo de regesion
X = vuelos_ventas.drop(columns=["TotalSales"])
y = vuelos_ventas["TotalSales"]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r111 = XGBRegressor(n_estimators=10, seed=123)
xgb_r = XGBRegressor(learning_rate=0.3, n_estimators=250)
xgb_r.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r.predict(x_test)

[0]	validation_0-rmse:2481.74894
[1]	validation_0-rmse:2198.97013
[2]	validation_0-rmse:2033.32798
[3]	validation_0-rmse:1907.75976
[4]	validation_0-rmse:1834.64862
[5]	validation_0-rmse:1773.85720
[6]	validation_0-rmse:1717.03975
[7]	validation_0-rmse:1690.80258
[8]	validation_0-rmse:1672.18555
[9]	validation_0-rmse:1654.92034
[10]	validation_0-rmse:1639.77394
[11]	validation_0-rmse:1630.80127
[12]	validation_0-rmse:1618.72050
[13]	validation_0-rmse:1610.92381
[14]	validation_0-rmse:1605.80254
[15]	validation_0-rmse:1602.88202
[16]	validation_0-rmse:1596.54765
[17]	validation_0-rmse:1590.12355
[18]	validation_0-rmse:1583.64605
[19]	validation_0-rmse:1581.56075
[20]	validation_0-rmse:1579.30407
[21]	validation_0-rmse:1577.54724
[22]	validation_0-rmse:1576.87235
[23]	validation_0-rmse:1571.83939
[24]	validation_0-rmse:1567.25605
[25]	validation_0-rmse:1565.76184
[26]	validation_0-rmse:1564.85253
[27]	validation_0-rmse:1562.18313
[28]	validation_0-rmse:1561.84044
[29]	validation_0-rmse:1

In [7]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.7157283740835436
Mean Absolute Error: 976.2706119747071
Mean Squared Error: 2366687.9791630553
Root Mean Squared Error: 1538.4043613962667


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Modelo cantidad**

In [8]:
# Merge a ambos dataframes
vuelos_ventas = pd.merge(ventas, vuelos, on="Flight_ID")
vuelos_ventas.drop(columns=["index", "Aeronave"], inplace=True)

In [9]:
vuelos_ventas["STD"] = pd.to_datetime(vuelos_ventas["STD"])
vuelos_ventas["STA"] = pd.to_datetime(vuelos_ventas["STA"])
vuelos_ventas["Duracion"] = vuelos_ventas["STA"] - vuelos_ventas["STD"]
vuelos_ventas["Duracion"] = vuelos_ventas["Duracion"].dt.total_seconds() / 60
vuelos_ventas["Hora"] = vuelos_ventas["STD"].dt.hour
vuelos_ventas["Dia"] = vuelos_ventas["STD"].dt.day
# Cambiar tipos de destinos a one-hot encoding
destination_type = vuelos_ventas["Destination_Type"].map(
    {
        "Ciudad Principal": "destino_ciudadprincipal",
        "Playa": "destino_playa",
        "MX Amigos y Familia": "destino_amigosfamilia",
        "Ciudad Fronteriza": "destino_ciudadfronteriza",
        "Ecoturismo": "destino_ecoturismo",
    }
)
destination_type = pd.get_dummies(destination_type, dtype="int")
vuelos_ventas = (
    pd.concat([vuelos_ventas, destination_type], axis=1)
    if "Destination_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
vuelos_ventas = (
    vuelos_ventas.drop(columns=["Destination_Type"])
    if "Destination_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
origen_type = vuelos_ventas["Origin_Type"].map(
    {
        "Ciudad Principal": "origen_ciudadprincipal",
        "Playa": "origen_playa",
        "MX Amigos y Familia": "origen_amigosfamilia",
        "Ciudad Fronteriza": "origen_ciudadfronteriza",
        "Ecoturismo": "origen_ecoturismo",
    }
)
origen_type = pd.get_dummies(origen_type, dtype="int")
vuelos_ventas = (
    pd.concat([vuelos_ventas, origen_type], axis=1)
    if "Origin_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
vuelos_ventas = (
    vuelos_ventas.drop(columns=["Origin_Type"])
    if "Origin_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
vuelos_ventas["Book_Pass"] = vuelos_ventas["Bookings"] / vuelos_ventas["Passengers"]
vuelos_ventas.drop(columns=["STD", "STA"], inplace=True)
vuelos_ventas = one_hot_encode(vuelos_ventas, "DepartureStation", "departure")
vuelos_ventas = one_hot_encode(vuelos_ventas, "ArrivalStation", "arrival")

In [10]:
vuelos_ventas.drop(columns=["Flight_ID", "ProductType", "TotalSales"], inplace=True)

In [11]:
product_dict = {}
for i in vuelos_ventas["ProductName"].unique():
    product_dict[i] = len(product_dict)
vuelos_ventas["ProductName"] = vuelos_ventas["ProductName"].map(product_dict)
product_dict_inv = {val: key for key, val in product_dict.items()}

In [12]:
# Encontrar los productos mas vendidos
product_sales = vuelos_ventas.groupby("ProductName")["Quantity"].sum()
product_sales = product_sales.sort_values(ascending=False)
top10 = product_sales.index[:10]
print([product_dict_inv[i] for i in top10])

['Coca Cola Regular', 'Agua Natural 600 Ml', 'Sabritas Originales', 'Ruffles Queso', 'Doritos Nacho', 'Panini Clasico', 'Fritos Limon Y Sal', 'Sabritas Flamin Hot', 'Sprite', 'Cheetos']


## **Coca**

In [13]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[0]]

In [14]:
vuelos_ventas.groupby("ProductName")["Quantity"].median()[9]

5.0

In [15]:
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r2 = XGBRegressor(n_estimators=10, seed=123)
xgb_r1 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r1.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r1.predict(x_test)

[0]	validation_0-rmse:5.25927
[1]	validation_0-rmse:4.83643
[2]	validation_0-rmse:4.66404
[3]	validation_0-rmse:4.57700
[4]	validation_0-rmse:4.52000
[5]	validation_0-rmse:4.47930
[6]	validation_0-rmse:4.44579
[7]	validation_0-rmse:4.42145
[8]	validation_0-rmse:4.41315
[9]	validation_0-rmse:4.39696
[10]	validation_0-rmse:4.38211
[11]	validation_0-rmse:4.37732
[12]	validation_0-rmse:4.37356
[13]	validation_0-rmse:4.37078
[14]	validation_0-rmse:4.36820
[15]	validation_0-rmse:4.36888
[16]	validation_0-rmse:4.36470
[17]	validation_0-rmse:4.35684
[18]	validation_0-rmse:4.35384
[19]	validation_0-rmse:4.35331
[20]	validation_0-rmse:4.34916
[21]	validation_0-rmse:4.34651
[22]	validation_0-rmse:4.34613
[23]	validation_0-rmse:4.34210
[24]	validation_0-rmse:4.34017
[25]	validation_0-rmse:4.33944
[26]	validation_0-rmse:4.33941
[27]	validation_0-rmse:4.33697
[28]	validation_0-rmse:4.33992
[29]	validation_0-rmse:4.33941
[30]	validation_0-rmse:4.33684
[31]	validation_0-rmse:4.33597
[32]	validation_0-

In [17]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print(
    "Root Mean Squared Error:",
    np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
)

R2 Score: 0.47195014894970644
Mean Absolute Error: 3.168193698105979
Mean Squared Error: 20.271020487713272
Root Mean Squared Error: 4.502335003941097


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Agua**

In [18]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[1]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r3 = XGBRegressor(n_estimators=10, seed=123)
xgb_r2 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r2.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r2.predict(x_test)

[0]	validation_0-rmse:4.30768
[1]	validation_0-rmse:4.04691
[2]	validation_0-rmse:3.91754
[3]	validation_0-rmse:3.84463
[4]	validation_0-rmse:3.79234
[5]	validation_0-rmse:3.75874
[6]	validation_0-rmse:3.71758
[7]	validation_0-rmse:3.70004
[8]	validation_0-rmse:3.67621
[9]	validation_0-rmse:3.66835
[10]	validation_0-rmse:3.65938
[11]	validation_0-rmse:3.64529
[12]	validation_0-rmse:3.64204
[13]	validation_0-rmse:3.63853
[14]	validation_0-rmse:3.61599
[15]	validation_0-rmse:3.61022
[16]	validation_0-rmse:3.60892
[17]	validation_0-rmse:3.60757
[18]	validation_0-rmse:3.60263
[19]	validation_0-rmse:3.60102
[20]	validation_0-rmse:3.60014
[21]	validation_0-rmse:3.59906
[22]	validation_0-rmse:3.59715
[23]	validation_0-rmse:3.59597
[24]	validation_0-rmse:3.59415
[25]	validation_0-rmse:3.59460
[26]	validation_0-rmse:3.59284
[27]	validation_0-rmse:3.58630
[28]	validation_0-rmse:3.58567
[29]	validation_0-rmse:3.58463
[30]	validation_0-rmse:3.58449
[31]	validation_0-rmse:3.58180
[32]	validation_0-

In [19]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.4365865655890262
Mean Absolute Error: 2.6145307317859143
Mean Squared Error: 13.794136460462491
Root Mean Squared Error: 3.7140458344590326


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Sabritas Originales**

In [20]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[2]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r4 = XGBRegressor(n_estimators=10, seed=123)
xgb_r3 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r3.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r3.predict(x_test)

[0]	validation_0-rmse:2.91804
[1]	validation_0-rmse:2.75313
[2]	validation_0-rmse:2.67716
[3]	validation_0-rmse:2.63141
[4]	validation_0-rmse:2.60938
[5]	validation_0-rmse:2.59443
[6]	validation_0-rmse:2.58884
[7]	validation_0-rmse:2.58248
[8]	validation_0-rmse:2.57777
[9]	validation_0-rmse:2.57351
[10]	validation_0-rmse:2.57328
[11]	validation_0-rmse:2.57013
[12]	validation_0-rmse:2.56850
[13]	validation_0-rmse:2.56779
[14]	validation_0-rmse:2.56734
[15]	validation_0-rmse:2.56630
[16]	validation_0-rmse:2.56713
[17]	validation_0-rmse:2.56775
[18]	validation_0-rmse:2.56782
[19]	validation_0-rmse:2.56895
[20]	validation_0-rmse:2.56828
[21]	validation_0-rmse:2.56505
[22]	validation_0-rmse:2.56335
[23]	validation_0-rmse:2.56416
[24]	validation_0-rmse:2.56410
[25]	validation_0-rmse:2.56403
[26]	validation_0-rmse:2.56509
[27]	validation_0-rmse:2.56528
[28]	validation_0-rmse:2.56511
[29]	validation_0-rmse:2.56535
[30]	validation_0-rmse:2.56691
[31]	validation_0-rmse:2.56656
[32]	validation_0-

In [21]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.3314282488274055
Mean Absolute Error: 1.9254828933585648
Mean Squared Error: 7.2951961908425185
Root Mean Squared Error: 2.700962086154213


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Ruffles Queso**

In [22]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[3]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r5 = XGBRegressor(n_estimators=10, seed=123)
xgb_r4 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r4.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r4.predict(x_test)

[0]	validation_0-rmse:2.52083
[1]	validation_0-rmse:2.36275
[2]	validation_0-rmse:2.30511
[3]	validation_0-rmse:2.27700
[4]	validation_0-rmse:2.25893
[5]	validation_0-rmse:2.24515
[6]	validation_0-rmse:2.23995
[7]	validation_0-rmse:2.23067
[8]	validation_0-rmse:2.22466
[9]	validation_0-rmse:2.21904
[10]	validation_0-rmse:2.21252
[11]	validation_0-rmse:2.21176
[12]	validation_0-rmse:2.21150
[13]	validation_0-rmse:2.21062
[14]	validation_0-rmse:2.21018
[15]	validation_0-rmse:2.20850
[16]	validation_0-rmse:2.20734
[17]	validation_0-rmse:2.20538
[18]	validation_0-rmse:2.20591
[19]	validation_0-rmse:2.20734
[20]	validation_0-rmse:2.20644
[21]	validation_0-rmse:2.20530
[22]	validation_0-rmse:2.20591
[23]	validation_0-rmse:2.20508
[24]	validation_0-rmse:2.20549
[25]	validation_0-rmse:2.20555
[26]	validation_0-rmse:2.20559
[27]	validation_0-rmse:2.20608
[28]	validation_0-rmse:2.20669
[29]	validation_0-rmse:2.20703
[30]	validation_0-rmse:2.20701
[31]	validation_0-rmse:2.20817
[32]	validation_0-

In [23]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.3468752555202571
Mean Absolute Error: 1.6364877067455268
Mean Squared Error: 5.375963540035936
Root Mean Squared Error: 2.318612416950262


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Doritos Nacho**

In [24]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[4]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r6 = XGBRegressor(n_estimators=10, seed=123)
xgb_r5 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r5.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r5.predict(x_test)

[0]	validation_0-rmse:2.38185
[1]	validation_0-rmse:2.28740
[2]	validation_0-rmse:2.23465
[3]	validation_0-rmse:2.20553
[4]	validation_0-rmse:2.18864
[5]	validation_0-rmse:2.17644
[6]	validation_0-rmse:2.16529
[7]	validation_0-rmse:2.15717
[8]	validation_0-rmse:2.15440
[9]	validation_0-rmse:2.15325
[10]	validation_0-rmse:2.14803
[11]	validation_0-rmse:2.14558
[12]	validation_0-rmse:2.14528
[13]	validation_0-rmse:2.14391
[14]	validation_0-rmse:2.13974
[15]	validation_0-rmse:2.14087
[16]	validation_0-rmse:2.14157
[17]	validation_0-rmse:2.14131
[18]	validation_0-rmse:2.14235
[19]	validation_0-rmse:2.14124
[20]	validation_0-rmse:2.14165
[21]	validation_0-rmse:2.14085
[22]	validation_0-rmse:2.14157
[23]	validation_0-rmse:2.14083
[24]	validation_0-rmse:2.14118
[25]	validation_0-rmse:2.14080
[26]	validation_0-rmse:2.13837
[27]	validation_0-rmse:2.13954
[28]	validation_0-rmse:2.13967
[29]	validation_0-rmse:2.13979
[30]	validation_0-rmse:2.13964
[31]	validation_0-rmse:2.13949
[32]	validation_0-

In [25]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.2696174940641838
Mean Absolute Error: 1.5986242804714084
Mean Squared Error: 5.120051293919524
Root Mean Squared Error: 2.262753034230542


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Panini Original**

In [26]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[5]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r7 = XGBRegressor(n_estimators=10, seed=123)
xgb_r6 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r6.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r6.predict(x_test)

[0]	validation_0-rmse:1.87736
[1]	validation_0-rmse:1.80173
[2]	validation_0-rmse:1.76571
[3]	validation_0-rmse:1.74993
[4]	validation_0-rmse:1.74083
[5]	validation_0-rmse:1.73133
[6]	validation_0-rmse:1.72662
[7]	validation_0-rmse:1.72525
[8]	validation_0-rmse:1.72412
[9]	validation_0-rmse:1.72198
[10]	validation_0-rmse:1.72189
[11]	validation_0-rmse:1.72162
[12]	validation_0-rmse:1.72061
[13]	validation_0-rmse:1.71767
[14]	validation_0-rmse:1.71747
[15]	validation_0-rmse:1.71759
[16]	validation_0-rmse:1.71777
[17]	validation_0-rmse:1.71767
[18]	validation_0-rmse:1.71729
[19]	validation_0-rmse:1.71746
[20]	validation_0-rmse:1.71754
[21]	validation_0-rmse:1.71737
[22]	validation_0-rmse:1.71699
[23]	validation_0-rmse:1.71695
[24]	validation_0-rmse:1.71656
[25]	validation_0-rmse:1.71776
[26]	validation_0-rmse:1.71964
[27]	validation_0-rmse:1.71917
[28]	validation_0-rmse:1.72029
[29]	validation_0-rmse:1.72062
[30]	validation_0-rmse:1.72064
[31]	validation_0-rmse:1.72089
[32]	validation_0-

In [27]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.24042592172159072
Mean Absolute Error: 1.3151072898881557
Mean Squared Error: 3.2973229894701706
Root Mean Squared Error: 1.8158532400692988


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Fritos Limon y Sal**

In [29]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[6]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r8 = XGBRegressor(n_estimators=10, seed=123)
xgb_r7 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r7.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r7.predict(x_test)

[0]	validation_0-rmse:1.95380
[1]	validation_0-rmse:1.87954
[2]	validation_0-rmse:1.84686
[3]	validation_0-rmse:1.83500
[4]	validation_0-rmse:1.82512
[5]	validation_0-rmse:1.81856
[6]	validation_0-rmse:1.81435
[7]	validation_0-rmse:1.81261
[8]	validation_0-rmse:1.81277
[9]	validation_0-rmse:1.81104
[10]	validation_0-rmse:1.80985
[11]	validation_0-rmse:1.80858
[12]	validation_0-rmse:1.80794
[13]	validation_0-rmse:1.80979
[14]	validation_0-rmse:1.80933
[15]	validation_0-rmse:1.80889
[16]	validation_0-rmse:1.80999
[17]	validation_0-rmse:1.80841
[18]	validation_0-rmse:1.80874
[19]	validation_0-rmse:1.80938
[20]	validation_0-rmse:1.81002
[21]	validation_0-rmse:1.81158
[22]	validation_0-rmse:1.81201
[23]	validation_0-rmse:1.81304
[24]	validation_0-rmse:1.81249
[25]	validation_0-rmse:1.81204
[26]	validation_0-rmse:1.81126
[27]	validation_0-rmse:1.81152
[28]	validation_0-rmse:1.81242
[29]	validation_0-rmse:1.81273
[30]	validation_0-rmse:1.81313
[31]	validation_0-rmse:1.81427
[32]	validation_0-

In [30]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.17595326188981086
Mean Absolute Error: 1.3605246169979748
Mean Squared Error: 3.7288391223934902
Root Mean Squared Error: 1.9310202283750137


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Sabritas Flamin Hot**

In [31]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[7]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r9 = XGBRegressor(n_estimators=10, seed=123)
xgb_r8 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r8.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r8.predict(x_test)

[0]	validation_0-rmse:1.97328
[1]	validation_0-rmse:1.90007
[2]	validation_0-rmse:1.85671
[3]	validation_0-rmse:1.82693
[4]	validation_0-rmse:1.81367
[5]	validation_0-rmse:1.79999
[6]	validation_0-rmse:1.79272
[7]	validation_0-rmse:1.78938
[8]	validation_0-rmse:1.78763
[9]	validation_0-rmse:1.78743
[10]	validation_0-rmse:1.78649
[11]	validation_0-rmse:1.78653
[12]	validation_0-rmse:1.78330
[13]	validation_0-rmse:1.78178
[14]	validation_0-rmse:1.78181
[15]	validation_0-rmse:1.78346
[16]	validation_0-rmse:1.78294
[17]	validation_0-rmse:1.78348
[18]	validation_0-rmse:1.78416
[19]	validation_0-rmse:1.78280
[20]	validation_0-rmse:1.78388
[21]	validation_0-rmse:1.78312
[22]	validation_0-rmse:1.78457
[23]	validation_0-rmse:1.78467
[24]	validation_0-rmse:1.78486
[25]	validation_0-rmse:1.78395
[26]	validation_0-rmse:1.78284
[27]	validation_0-rmse:1.78353
[28]	validation_0-rmse:1.78237
[29]	validation_0-rmse:1.78428
[30]	validation_0-rmse:1.78494
[31]	validation_0-rmse:1.78571
[32]	validation_0-

In [32]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.21887666898772495
Mean Absolute Error: 1.3324718879996902
Mean Squared Error: 3.628609556047475
Root Mean Squared Error: 1.9048909564716494


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Sprite**

In [33]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[8]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r10 = XGBRegressor(n_estimators=10, seed=123)
xgb_r9 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r9.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r9.predict(x_test)

[0]	validation_0-rmse:1.61823
[1]	validation_0-rmse:1.55892
[2]	validation_0-rmse:1.53629
[3]	validation_0-rmse:1.52659
[4]	validation_0-rmse:1.51945
[5]	validation_0-rmse:1.51438
[6]	validation_0-rmse:1.51335
[7]	validation_0-rmse:1.51249
[8]	validation_0-rmse:1.51103
[9]	validation_0-rmse:1.51037
[10]	validation_0-rmse:1.50994
[11]	validation_0-rmse:1.51025
[12]	validation_0-rmse:1.50996
[13]	validation_0-rmse:1.51012
[14]	validation_0-rmse:1.50977
[15]	validation_0-rmse:1.50961
[16]	validation_0-rmse:1.50946
[17]	validation_0-rmse:1.50853
[18]	validation_0-rmse:1.50884
[19]	validation_0-rmse:1.50960
[20]	validation_0-rmse:1.51069
[21]	validation_0-rmse:1.51060
[22]	validation_0-rmse:1.51069
[23]	validation_0-rmse:1.51115
[24]	validation_0-rmse:1.51136
[25]	validation_0-rmse:1.51192
[26]	validation_0-rmse:1.51292
[27]	validation_0-rmse:1.51336
[28]	validation_0-rmse:1.51334
[29]	validation_0-rmse:1.51317
[30]	validation_0-rmse:1.51430
[31]	validation_0-rmse:1.51620
[32]	validation_0-

In [34]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.1458556586776849
Mean Absolute Error: 1.1675276020336685
Mean Squared Error: 2.6295457662892248
Root Mean Squared Error: 1.6215874217226849


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

## **Cheetos**

In [35]:
modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == top10[8]]
X = modelo2.drop(columns=["Quantity"])
y = modelo2["Quantity"]
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r10 = XGBRegressor(n_estimators=10, seed=123)
xgb_r10 = XGBRegressor(learning_rate=0.5, n_estimators=250)
xgb_r10.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r10.predict(x_test)

[0]	validation_0-rmse:1.61823
[1]	validation_0-rmse:1.55892
[2]	validation_0-rmse:1.53629
[3]	validation_0-rmse:1.52659
[4]	validation_0-rmse:1.51945
[5]	validation_0-rmse:1.51438
[6]	validation_0-rmse:1.51335
[7]	validation_0-rmse:1.51249
[8]	validation_0-rmse:1.51103
[9]	validation_0-rmse:1.51037
[10]	validation_0-rmse:1.50994
[11]	validation_0-rmse:1.51025
[12]	validation_0-rmse:1.50996
[13]	validation_0-rmse:1.51012
[14]	validation_0-rmse:1.50977
[15]	validation_0-rmse:1.50961
[16]	validation_0-rmse:1.50946
[17]	validation_0-rmse:1.50853
[18]	validation_0-rmse:1.50884
[19]	validation_0-rmse:1.50960
[20]	validation_0-rmse:1.51069
[21]	validation_0-rmse:1.51060
[22]	validation_0-rmse:1.51069
[23]	validation_0-rmse:1.51115
[24]	validation_0-rmse:1.51136
[25]	validation_0-rmse:1.51192
[26]	validation_0-rmse:1.51292
[27]	validation_0-rmse:1.51336
[28]	validation_0-rmse:1.51334
[29]	validation_0-rmse:1.51317
[30]	validation_0-rmse:1.51430
[31]	validation_0-rmse:1.51620
[32]	validation_0-

In [36]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.1458556586776849
Mean Absolute Error: 1.1675276020336685
Mean Squared Error: 2.6295457662892248
Root Mean Squared Error: 1.6215874217226849


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is

In [39]:
# import pickle

# # Supongamos que tienes una lista de nombres de modelos
# nombres_modelos = [
#     "ModeloVentas",
#     "ModeloCoca",
#     "ModeloAgua",
#     "ModeloSabritas",
#     "ModeloRuffles",
#     "ModeloDoritos",
#     "ModeloPanini",
#     "ModeloFritos",
#     "ModeloFlamin",
#     "ModeloSprite",
#     "ModeloCheetos",
# ]

# # Supongamos también que tienes una lista de modelos ya entrenados
# modelos_entrenados = [
#     xgb_r,
#     xgb_r1,
#     xgb_r2,
#     xgb_r3,
#     xgb_r4,
#     xgb_r5,
#     xgb_r6,
#     xgb_r7,
#     xgb_r8,
#     xgb_r9,
#     xgb_r10,
# ]

# # Guardar cada modelo en un archivo pickle con su nombre correspondiente
# for nombre, modelo in zip(nombres_modelos, modelos_entrenados):
#     with open(f"{nombre}.pkl", "wb") as archivo:
#         pickle.dump(modelo, archivo)

In [ ]:
# import pickle

# for i in range(93, 98):
#     modelo2 = vuelos_ventas[vuelos_ventas["ProductName"] == i]
#     X = modelo2.drop(columns=["Quantity"])
#     y = modelo2["Quantity"]
#     x_train, x_test, y_train, y_test = train_test_split(
#         X,
#         y,
#         test_size=0.3,
#         random_state=0,
#     )

#     eval_set = [(x_test, y_test)]

#     train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
#     test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

#     xgb_r = XGBRegressor(learning_rate=0.5, n_estimators=250)
#     xgb_r.fit(x_train, y_train, eval_set=eval_set, verbose=True)
#     y_pred = xgb_r.predict(x_test)

#     # Evaluamos el modelo
#     print("R2 Score:", metrics.r2_score(y_test, y_pred))
#     print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
#     print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
#     print(
#         "Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred))
#     )

#     with open(f"Modelo{i}.pkl", "wb") as archivo:
#         pickle.dump(xgb_r, archivo)

In [58]:
# Faltan 88 y 92

,ProductName,Quantity,Capacity,Passengers,Bookings,Semana,Mes,Duracion,Hora,Dia,...,arrival_BJ,arrival_BK,arrival_BL,arrival_BM,arrival_BN,arrival_BO,arrival_BP,arrival_BQ,arrival_BS,arrival_BT
